#### импорты

In [1]:
from credentials import dbname, host, user, password, host, port
import pandas as pd
from sqlalchemy import create_engine

#### загрузка данных из БД

In [2]:
# with create_engine(f'postgresql://{user}@{host}:{port}/{dbname}').connect() as connection:
with create_engine('postgresql://postgres@localhost:5432/postgres').connect() as connection:
    pd.read_sql("select * from public.transformed_raw as t", connection).to_csv('t.csv', index=False)

#### загрузка данных с диска

In [4]:
df = pd.read_csv('t.csv', index_col='id')
df

,application_datetime,MTS_score,fraud_score,OKB_score,antibankrupt_score,Unnamed: 0,avg_credit_card_utilization_last_12m,num_late_payments_last_24m,max_loan_amount_past_5y,num_active_loans,...,ekapusta_vector_9,Rusinterfinance_vector,gamma_refactor_pod,quanta_latch_jx,hexmux_torque_sr,delta_noise_pin,polysync_emitter_vx,wrangle_bitport_8,trace_mux_qr,scramble_node_5
id,,,,,,,,,,,,,,,,,,,,,
4633210,2024-01-01 00:12:37,0.056,0.324,0.20122,0.75,1088,NaN,0.062,NaN,0.470,...,0.886478,-0.810528,-0.442521,-0.138927,-1.203383,-0.707538,-0.975481,0.824046,-0.483414,-0.477379
4633211,2024-01-01 00:44:13,0.575,0.192,0.51945,0.70,-1893,0.000,0.691,NaN,1.000,...,-1.197664,-0.501757,-0.677516,-1.389196,-0.419252,-0.403336,-1.856283,0.136344,2.202533,0.511516
4633212,2024-01-01 00:53:24,0.956,0.247,0.24840,0.79,424,0.000,0.776,NaN,0.941,...,-0.216561,1.162716,2.237123,-1.446566,-1.557770,-0.533780,1.146970,-0.786685,-0.010330,0.957952
4633213,2024-01-01 01:04:00,0.563,0.452,0.39770,0.70,1822,0.000,0.724,0.000,1.000,...,2.174569,0.691166,0.524389,-1.287966,0.064069,-1.728057,-0.357496,-0.162440,0.410391,-1.452951
4633214,2024-01-01 01:55:44,0.650,0.505,0.34203,0.72,-2016,0.000,0.895,0.000,0.940,...,0.676933,1.647065,0.316818,-0.382999,0.513072,0.132060,1.333927,-0.002653,-0.131036,-0.054672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643205,2024-04-30 21:23:28,0.135,0.735,0.08059,0.30,8667,0.000,0.876,NaN,0.628,...,-1.159872,0.604449,0.616036,0.310525,0.650636,1.607563,-0.716779,0.943228,-1.046344,-0.597491
4643206,2024-04-30 21:54:10,0.578,0.915,0.08951,0.96,8012,0.000,0.433,0.000,0.327,...,0.134771,-0.907361,-1.971117,0.642404,-0.131189,0.746628,-0.781307,-1.202640,0.208824,-0.213604
4643207,2024-04-30 22:12:55,0.322,0.774,0.01174,0.97,11332,0.000,0.732,0.000,1.000,...,0.221629,-0.683927,-0.396360,-1.604916,0.657322,-0.087542,0.009548,0.744417,-0.638447,-0.776942


In [44]:
threshold_nan = 0.9
threshold_value_count = 0.8
threshold_importance = 0.01
threshold_corr = 0.5

In [9]:
X_columns = df.drop(columns='def_45').columns
len(X_columns)

388

#### удаляем колонки с пропусками

In [18]:
X_columns_1 = [col for col in X_columns if df[col].isna().mean() <= threshold_nan]
len(X_columns_1)

184

#### удалим колонки где много одинаковых значений

In [25]:
X_columns_2 = [col for col in X_columns_1 if (df[col].value_counts(normalize=True, dropna=False).max() <= threshold_value_count)]
len(X_columns_2)

175

#### катбуст с регуляризацией

In [ ]:
df['application_datetime'].max()

'2024-04-30 23:56:08'

In [29]:
split_date = '2024-04-01 01:55:44'

df_train = df[df['application_datetime'] < split_date]
df_test = df[df['application_datetime'] > split_date]
len(df_test) / len (df)

0.2463

In [34]:
X_train = df_train[X_columns_2].select_dtypes(include=["float64", "int64"])
y_train = df_train['def_45']

In [ ]:
from catboost import CatBoostClassifier

params = {
    "l2_leaf_reg": 10,
    "random_state": 42,
    "iterations": 300,
    "max_depth": 4,
}

model = CatBoostClassifier(**params)
model.fit(X_train, y_train)
importances = pd.Series(model.get_feature_importance(), index=model.feature_names_)

In [43]:
X_columns_3 = list(importances[importances >= threshold_importance].index)
len(X_columns_3)

63

#### фильтр корреляций

In [49]:
from tqdm import tqdm

In [55]:
corr_matrix = df[X_columns_3].corr().abs()

to_drop = set()

for i in tqdm(range(len(X_columns_3))):
    for j in range(i+1, len(X_columns_3)):
        col_i = X_columns_3[i]
        col_j = X_columns_3[j]
        if corr_matrix.loc[col_i, col_j] > threshold_corr:
            if importances[col_i] > importances[col_j]:
                to_drop.add(col_i)
            else:
                to_drop.add(col_j)

X_columns_4 = [col for col in X_columns_3 if col not in to_drop]
len(X_columns_4)

100%|██████████| 63/63 [00:00<00:00, 2396.79it/s]


41

In [ ]:
corr_matrix

In [57]:
df[[X_columns_4[0], 'application_datetime']]

,MTS_score,application_datetime
id,,
4633210,0.056,2024-01-01 00:12:37
4633211,0.575,2024-01-01 00:44:13
4633212,0.956,2024-01-01 00:53:24
4633213,0.563,2024-01-01 01:04:00
4633214,0.650,2024-01-01 01:55:44
...,...,...
4643205,0.135,2024-04-30 21:23:28
4643206,0.578,2024-04-30 21:54:10
4643207,0.322,2024-04-30 22:12:55


#### обучаем модель

In [58]:
from sklearn.metrics import roc_auc_score

In [61]:
X_train = df_train[X_columns_4]
X_test = df_test[X_columns_4]

y_train = df_train['def_45']
y_test = df_test['def_45']

In [ ]:
model = CatBoostClassifier()
model.fit(X_train, y_train)

In [69]:
def gini(y_true, y_pred):
    return 2*roc_auc_score(y_true, y_pred) - 1

In [66]:
y_pred = model.predict_proba(X_test)[:,1]

In [67]:
y_pred

array([9.98556448e-01, 9.99970794e-01, 9.99677786e-01, ...,
       3.71863489e-05, 1.06085009e-04, 6.77969551e-04], shape=(2463,))

In [70]:
gini(y_test, y_pred)

0.9971618020697777

In [71]:
df['application_datetime'] = pd.to_datetime(df['application_datetime'])

In [72]:
import matplotlib.pyplot as plt

In [ ]:
for col in tqdm(X_columns_4):
    df['binned_col'] = pd.cut(df[col], bins=10)
    grouped = df.groupby([df['application_datetime'].dt.to_period('W'), 'binned_col'], observed=True).size().unstack(fill_value=0)

    grouped = grouped.div(grouped.sum(axis=1), axis=0)

    plt.figure(figsize=(16, 6))
    if not grouped.empty and grouped.select_dtypes(include='number').shape[1] > 0:
        ax = grouped.plot(kind='bar', stacked=True, width=1, cmap='viridis', ax=plt.gca())
        ax.set_xticks(ax.get_xticks()[::])
    plt.xlabel('Week')
    plt.ylabel('Proportion')
    plt.title(f'Weekly Proportional Distribution of {col}')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks()
    plt.tight_layout()
    plt.show()

In [ ]:
for col in tqdm(X_columns_4):
    df['binned_col'], bins = pd.qcut(df[col], q=20, retbins=True, duplicates='drop')
    grouped = df.groupby([df['application_datetime'].dt.to_period('W'), 'binned_col'], observed=True).size().unstack(fill_value=0)

    grouped = grouped.div(grouped.sum(axis=1), axis=0)

    plt.figure(figsize=(16, 6))
    if not grouped.empty and grouped.select_dtypes(include='number').shape[1] > 0:
        ax = grouped.plot(kind='bar', stacked=True, width=1, cmap='viridis', ax=plt.gca())
        ax.set_xticks(ax.get_xticks()[::])
    plt.xlabel('Week')
    plt.ylabel('Proportion')
    plt.title(f'Weekly Proportional Distribution of {col}')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks()
    plt.tight_layout()
    plt.show()

In [ ]:
df_test['pred'] = y_pred
df_test

In [82]:
len(df_test[df_test['pred']<0.1]) / len(df_test)

0.6366220056841251

In [83]:
df_test['score'] = (1-df_test['pred']) * 1000
df_test[['pred', 'score']]

/var/folders/1f/wsx4ksrj6cl_lfr199q7lm800000gn/T/ipykernel_34156/2492693331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['score'] = (1-df_test['pred']) * 1000


,pred,score
id,,
4636072,0.998556,1.443552
4636073,0.999971,0.029206
4636074,0.999678,0.322214
4636075,0.793609,206.391240
4636076,0.976056,23.943965
...,...,...
4643205,0.000071,999.929201
4643206,0.000136,999.864167
4643207,0.000037,999.962814


In [91]:
cutoff = 900

In [92]:
df_test['RESOLUTION'] = df_test['score']>cutoff
df_test[['pred', 'score', 'RESOLUTION']]

/var/folders/1f/wsx4ksrj6cl_lfr199q7lm800000gn/T/ipykernel_34156/2785705589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['RESOLUTION'] = df_test['score']>cutoff


,pred,score,RESOLUTION
id,,,
4636072,0.998556,1.443552,False
4636073,0.999971,0.029206,False
4636074,0.999678,0.322214,False
4636075,0.793609,206.391240,False
4636076,0.976056,23.943965,False
...,...,...,...
4643205,0.000071,999.929201,True
4643206,0.000136,999.864167,True
4643207,0.000037,999.962814,True


In [99]:
import numpy as np


cutoffs = np.linspace(0, 1000, 50)
ars = [len(df_test[df_test['score']>cutoff])/len(df_test) for cutoff in cutoffs]

In [100]:
df_result = pd.DataFrame(data = {"cutoff":cutoffs, "одобрение":ars})

In [101]:
df_result

,cutoff,одобрение
0,0.000000,1.000000
1,20.408163,0.809988
2,40.816327,0.771417
3,61.224490,0.746650
4,81.632653,0.734876
5,102.040816,0.724320
6,122.448980,0.714982
7,142.857143,0.712952
8,163.265306,0.708080
9,183.673469,0.701583


в реальности, одобрение считается по недавней выборке всех заявок, а не по выборке выданных кредитов как это делаем мы, но это для примера. Также, тут не хватает ожидаемой просрочки, но её можно оценить только примерно.